In [1]:
using Revise

using Test
import TensorCrossInterpolation as TCI
import Random

In [2]:
using Test
import TensorCrossInterpolation as TCI
import TensorCrossInterpolation: rank, linkdims, TensorCI2, MultiIndex, evaluate, crossinterpolate2, pivoterror, tensortrain
import Random
import QuanticsGrids as QD

In [3]:
import TensorCrossInterpolation as TCI
using TensorCrossInterpolation
using Random
using Test
using ITensors
ITensors.disable_warn_order()

14

In [26]:
seed = 124567890

Random.seed!(seed)

R = 12
sites = [Index(2, "n=$n") for n in 1:R]

Ψ = random_mps(sites; linkdims=20)
rindex = Set([rand(1:2, R) for _ in 1:20])

tensor = Array(reduce(*, Ψ), reverse(sites))
tensor = tensor ./ maximum(abs, tensor)

f(x) = tensor[x...]

localdims = fill(2, R)
initialpivots = MultiIndex[]
for _ in 1:200
    push!(initialpivots, [rand(1:d) for d in localdims])
end
#push!(initialpivots, [1, 1, 1, 1])
#push!(initialpivots, [2, 1, 1, 1])
#push!(initialpivots, [2, 1, 1, 1])
#push!(initialpivots, [2, 2, 1, 1])

tci = TensorCI2{Float64}(f, localdims, initialpivots)

TensorCI2{Float64}([[1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2], [2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1], [1, 1, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1], [2, 1, 1, 1, 2, 1, 2, 1, 2, 2, 1, 1], [1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 1, 2], [2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1], [2, 1, 1, 1, 2, 2, 2, 1, 2, 1, 1, 1], [1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2], [1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 1, 1], [1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2]  …  [1, 2, 1, 1, 1, 2, 1, 2, 2, 1, 1, 2], [2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2], [1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2], [2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 2], [2, 1, 2, 2, 1, 2, 1, 2, 1, 1, 1, 1], [1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1], [1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 2, 1], [1, 2, 1, 2, 2, 2, 1, 1, 2, 1, 2, 2], [1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2], [2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1]], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], Float64[], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 0.4568748294361369)

In [27]:
sitetensors, Iset, Jset = TCI.sitetensors_site0update(tci, f)
tt = TCI.TensorTrain(sitetensors)

(::TensorTrain{Float64, 3}) (generic function with 1 method)

In [28]:
function checknesting2(L, Iset, Jset, pivot)
    leftnested = fill(false, L-1)
    rightnested = fill(false, L-1)
    for b in 1:L-1
        if pivot[1:b] ∈ Iset[b+1]
            leftnested[b] = true
        end
    end
    for b in reverse(1:L-1)
        if pivot[b+1:end] ∈ Jset[b]
            rightnested[b] = true
        end
    end
    return leftnested, rightnested
end
#for i in initialpivots
    #@show i, checknesting2(R, Iset, Jset, i)
#end

checknesting2 (generic function with 1 method)

In [29]:
function checknesting(L, Iset, Jset, pivot)
    leftnested = fill(false, L-1)
    rightnested = fill(false, L-1)
    for b in 1:L-1
        if pivot[1:b] ∈ Iset[b+1]
            leftnested[b] = true
        else
            break
        end
    end
    for b in reverse(1:L-1)
        if pivot[b+1:end] ∈ Jset[b]
            rightnested[b] = true
        else
            break
        end
    end

    fullnestedbond = Int[]
    for b in 1:L-1
        if leftnested[b] && rightnested[b]
            push!(fullnestedbond, b)
        end
    end
    return fullnestedbond
end

checknesting (generic function with 1 method)

In [32]:
tt = TCI.TensorTrain(TCI.sitetensors(tci, f))

for b in 1:length(tci)-1
    for i in Iset[b+1], j in Jset[b]
        p = vcat(i, j)
        if abs(TCI.evaluate(tt, p) - f(p)) > 1e-10
            @show b, i, j
        end
        #res = checknesting(length(tci), Iset, Jset, p)
        #if isempty(res)
            #@show "not nested", b, i, j
            #@show abs(TCI.evaluate(tt, p) - f(p))
        #end
    end
end

orthocenter = 6


In [31]:
for p in initialpivots
    bs = checknesting(R, Iset, Jset, p)
    println("$p $bs")
    if isempty(bs)
        @error "$p is not full nested at any bond!"
    end
end

[1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2] Int64[]
[2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1] [5, 6]
[1, 1, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1] [6]
[2, 1, 1, 1, 2, 1, 2, 1, 2, 2, 1, 1] [5, 6, 7, 8]
[1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 1, 2] [5, 6]
[2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1] [3, 4, 5, 6]
[2, 1, 1, 1, 2, 2, 2, 1, 2, 1, 1, 1] [6]
[1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2] [5, 6]
[1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 1, 1] Int64[]
[1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2] [6]
[1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1] [6, 7]
[1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 2] [6, 7]
[1, 1, 2, 1, 2, 2, 2, 2, 1, 1, 1, 1] [6]
[2, 1, 2, 2, 1, 1, 2, 2, 1, 2, 1, 1] [5, 6]
[1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1] [5, 6]
[1, 1, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2] [6]
[1, 1, 2, 2, 1, 1, 1, 2, 2, 1, 1, 2] [5, 6]
[1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2] [6]
[1, 1, 1, 2, 2, 2, 2, 1, 2, 1, 2, 1] [6]
[1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 1, 2] [6]
[2, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1] [6]
[1, 2, 1, 1, 2, 1, 2, 2, 1, 2, 2, 1] [6]
[1, 2, 2, 2, 2, 1, 1, 1, 2, 2, 1, 1] [5, 6]
[1, 

┌ Error: [1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2] is not full nested at any bond!
└ @ Main /Users/hiroshi/git/TensorCrossInterpolation.jl/notebook/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W6sZmlsZQ==.jl:5
┌ Error: [1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 1, 1] is not full nested at any bond!
└ @ Main /Users/hiroshi/git/TensorCrossInterpolation.jl/notebook/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W6sZmlsZQ==.jl:5
┌ Error: [1, 2, 1, 2, 2, 1, 1, 2, 1, 2, 1, 2] is not full nested at any bond!
└ @ Main /Users/hiroshi/git/TensorCrossInterpolation.jl/notebook/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W6sZmlsZQ==.jl:5
┌ Error: [1, 2, 1, 2, 2, 1, 2, 1, 1, 2, 2, 2] is not full nested at any bond!
└ @ Main /Users/hiroshi/git/TensorCrossInterpolation.jl/notebook/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W6sZmlsZQ==.jl:5
┌ Error: [1, 1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2] is not full nested at any bond!
└ @ Main /Users/hiroshi/git/TensorCrossInterpolation.jl/notebook/jl_notebook_cell_

In [56]:
for b in 1:length(tci)-1
    @show b
    @show Iset[b+1]
    @show Jset[b]
    println("")
end

b = 1
Iset[b + 1] = [[2], [1]]
Jset[b] = [[2, 2, 1, 1, 2, 1, 1], [1, 2, 1, 2, 1, 1, 2]]

b = 2
Iset[b + 1] = [[2, 2], [1, 1], [2, 1], [1, 2]]
Jset[b] = [[2, 1, 1, 2, 1, 1], [2, 1, 2, 1, 1, 2], [1, 1, 2, 1, 2, 1], [1, 2, 2, 2, 2, 2]]

b = 3
Iset[b + 1] = [[1, 2, 2], [2, 2, 2], [1, 1, 2], [1, 2, 1], [2, 1, 1], [1, 1, 1], [2, 2, 1], [2, 1, 2]]
Jset[b] = [[1, 1, 1, 1, 2], [1, 1, 2, 1, 1], [1, 2, 1, 1, 2], [2, 2, 2, 2, 2], [1, 1, 1, 2, 2], [2, 1, 1, 1, 2], [2, 1, 1, 2, 2], [2, 1, 1, 2, 1]]

b = 4
Iset[b + 1] = [[1, 2, 2, 1], [1, 1, 1, 2], [1, 1, 2, 1], [2, 2, 2, 1], [2, 1, 2, 2], [2, 2, 2, 2], [2, 2, 1, 2], [1, 2, 1, 2], [2, 2, 1, 1], [2, 1, 2, 1], [2, 1, 1, 1], [1, 2, 1, 1], [1, 1, 2, 2], [1, 2, 2, 2], [1, 1, 1, 1], [2, 1, 1, 2]]
Jset[b] = [[1, 1, 1, 2], [2, 1, 1, 2], [1, 2, 1, 1], [2, 1, 2, 2], [1, 1, 2, 1], [1, 1, 1, 1], [1, 1, 2, 2], [2, 2, 2, 2], [1, 2, 2, 2], [2, 2, 1, 1], [2, 1, 2, 1], [1, 2, 2, 1], [2, 2, 1, 2], [1, 2, 1, 2], [2, 2, 2, 1], [2, 1, 1, 1]]

b = 5
Iset[b + 1] = [[1, 2, 

In [57]:
#pivot = [2, 2, 1, 1]
pivot = [1, 1, 1, 1]
sitetensors_p = [t[:, p, :] for (p, t) in zip(pivot, tt)]
for t in sitetensors_p
    @show size(t)
end

size(t) = (1, 2)
size(t) = (2, 4)
size(t) = (4, 8)
size(t) = (8, 16)
